# Sample ARC Submission

This is a sample notebook that can help get you started with creating an ARC Prize submission. It covers the basics of loading libraries, loading data, implementing an approach, and submitting.

You should be able to submit this notebook to the evaluation portal and have it run successfully (although you'll get a score of 0, so you'll need to do some work if you want to do better!)


# Load needed libraries

Basic libraries like numpy, torch, matplotlib, and tqdm are already installed.

In [1]:
import json
from tqdm import tqdm

In [2]:
from random import choice

# Load the data

Here we are loading the training challenges and solutions (this is the public training set), the evaluation challenges and solutions (this is the public evaluation set), and the test challenges (currently a placeholder file that is a copy of the public evaluation challanges).

For your initial testing and exploration, I'd recommend not using the public evaluation set, just work off the public training set and then test against the test challenges (which is actually the public evaluation set). However, when competing in the competition, then you can should probably use the evaluation tasks for training too.

In [3]:
# Public training set
train_challenges_path = '../input/arc-prize-2024/arc-agi_training_challenges.json'
train_solutions_path = '../input/arc-prize-2024/arc-agi_training_solutions.json'

with open(train_challenges_path) as fp:
    train_challenges = json.load(fp)
with open(train_solutions_path) as fp:
    train_solutions = json.load(fp)

# Public evaluation set
evaluation_challenges_path = '../input/arc-prize-2024/arc-agi_training_challenges.json'
evaluation_solutions_path = '../input/arc-prize-2024/arc-agi_training_solutions.json'

with open(evaluation_challenges_path) as fp:
    evaluation_challenges = json.load(fp)
with open(evaluation_solutions_path) as fp:
    evaluation_solutions = json.load(fp)

# This will be the hidden test challenges (currently has a placeholder to the evaluation set)
test_challenges_path = '../input/arc-prize-2024/arc-agi_test_challenges.json'

with open(test_challenges_path) as fp:
    test_challenges = json.load(fp)

Here is an example of what a test task looks like:

In [4]:
sample_task = list(test_challenges.keys())[0]
challenge_ex = test_challenges[sample_task]
challenge_ex
# sample_task

{'test': [{'input': [[3, 2], [7, 8]]}],
 'train': [{'input': [[8, 6], [6, 4]],
   'output': [[8, 6, 8, 6, 8, 6],
    [6, 4, 6, 4, 6, 4],
    [6, 8, 6, 8, 6, 8],
    [4, 6, 4, 6, 4, 6],
    [8, 6, 8, 6, 8, 6],
    [6, 4, 6, 4, 6, 4]]},
  {'input': [[7, 9], [4, 3]],
   'output': [[7, 9, 7, 9, 7, 9],
    [4, 3, 4, 3, 4, 3],
    [9, 7, 9, 7, 9, 7],
    [3, 4, 3, 4, 3, 4],
    [7, 9, 7, 9, 7, 9],
    [4, 3, 4, 3, 4, 3]]}]}

# Generating a submission

To generate a submission you need to output a file called `submission.json` that has the following format:

```
{"00576224": [{"attempt_1": [[0, 0], [0, 0]], "attempt_2": [[0, 0], [0, 0]]}],
 "009d5c81": [{"attempt_1": [[0, 0], [0, 0]], "attempt_2": [[0, 0], [0, 0]]}],
 "12997ef3": [{"attempt_1": [[0, 0], [0, 0]], "attempt_2": [[0, 0], [0, 0]]},
              {"attempt_1": [[0, 0], [0, 0]], "attempt_2": [[0, 0], [0, 0]]}],
 ...
}
```

In this case, the task ids come from `test_challenges`. There may be multiple (i.e., >1) test items per task. Therefore, the dictionary has a list of dicts for each task. These submission dictionaries should appear in the same order as the test items from `test_challenges`. Additionally, you can provide two attempts for each test item. In fact, you **MUST** provide two attempts. If you only want to generate a single attempt, then just submit the same answer for both attempts (or submit an empty submission like the ones shown in the example snippit just above).

In [5]:
## BORROWED CODE: BEGIN CODE FROM HW 2 STARTER CODE

class ARCPuzzle:
    """
    A class to represent the state and the possible transformation
    actions of an ARC puzzle or partially transformed ARC puzzle
    """

    def __init__(self, state):
        self.state = tuple((tuple(row) for row in state))

    def __hash__(self):
        return hash(self.state)

    def __eq__(self, other):
        if isinstance(other, ARCPuzzle):
            return self.state == other.state
        return False

    def __ne__(self, other):
        return not self.__eq__(other)

    def __repr__(self):
        return str(self)

    def __str__(self):
        out = ""
        for row in self.state:
            out += str(row) + "\n"
        return out

    def copy(self):
        """
        Makes a deep copy of an ARCPuzzle object.
        """
        new = ARCPuzzle(self.state)
        return new

    def randomize(self, num_shuffles):
        """
        Randomizes an ARCPuzzle by executing a random action `num_suffles`
        times.
        """
        for i in range(num_shuffles):
            actions = [a for a in self.legalActions()]
            a = choice(actions)
            # print(actions, a)
            self.executeAction(a)

        return self

    def tophalf(self, grid):
        """ upper half """
        return grid[:len(grid) // 2]
    
    def bottomhalf(self, grid):
        """ lower half """
        return grid[len(grid) // 2:]


    def rot90(self, grid):
        """ clockwise rotation by 90 degrees """
        return tuple(zip(*grid[::-1]))


    def hmirror(self, grid):
        """ mirroring along horizontal """
        if len(grid) < 1:
            return grid
        if isinstance(grid[0], tuple):
            return tuple(row[::-1] for row in grid)
        else:
            return grid[::-1]

    def vmirror(self, grid):
        """ mirroring along vertical """
        return tuple(reversed(grid))

    def lshift(self, grid):
        return tuple([tuple([e for e in row if e != 0] + [0]*row.count(0))
                      for row in grid])

    def compress(self, grid):
        """ removes frontiers """
        ri = [i for i, r in enumerate(grid) if len(set(r)) == 1]
        ci = [j for j, c in enumerate(zip(*grid)) if len(set(c)) == 1]
        return tuple([tuple([v for j, v in enumerate(r) if j not in ci])
                      for i, r in enumerate(grid) if i not in ri])

    def mapcolor(self, grid, a, b):
        return tuple(tuple(b if e == a else e for e in row) for row in grid)

    def trim(self, grid):
        """ removes border """
        return tuple(r[1:-1] for r in grid[1:-1])

    def executeAction(self, action):
        """
        Executes an action to the ARCPuzzle object.

        :param action: the action to execute
        :type action: "up", "left", "right", or "down"
        """
        if action == 'tophalf':
            self.state = self.tophalf(self.state)
        elif action == 'bottomhalf':
            self.state = self.bottomhalf(self.state)
        elif action == 'rot90':
            self.state = self.rot90(self.state)
        elif action == 'hmirror':
            self.state = self.hmirror(self.state)
        elif action == 'vmirror':
            self.state = self.vmirror(self.state)
        elif action == 'lshift':
            self.state = self.lshift(self.state)
        elif action == 'compress':
            self.state = self.compress(self.state)
        elif action[:8] == 'mapcolor':
            args = action[9:-1].split(',')
            self.state = self.mapcolor(self.state, int(args[0]), int(args[1]))
        else:
            raise RuntimeError("Not an action")

    ## color space is the set of colors present in either the input or the output image
    def legalActions(self, color_space=set(range(10))):
        """
        Returns an iterator to the legal actions that can be executed in the
        current state.
        """
        action_search_space = ['tophalf', 'bottomhalf', 'hmirror', 'vmirror', 'lshift', 'compress'] # ['tophalf', 'rot90', 'hmirror', 'vmirror', 'lshift', 'compress']

        for action in action_search_space:
            yield action

        for a in set(e for row in self.state for e in row):
            for b in color_space:
                if a == b:
                    continue
                yield f'mapcolor({a},{b})'

## BORROWED CODE: END CODE FROM HW 2 STARTER CODE


In [6]:
def arc_search(input_grid, output_grid, max_depth=5) -> list[str]:
    """
    Searches for a sequence of transformations to convert the input_grid into
    the output_grid.
    
    Parameters:
    - input_grid: The initial grid that needs to be transformed.
    - output_grid: The target grid to achieve.
    - available_operations: List of primitive operations that can be used to transform the input grid.
    
    Returns:
    - A list of actions that transforms input_grid into output_grid.
    """
    
    # set of all colors present in this problem
    combined_list = [num for row in input_grid for num in row] + [num for row in output_grid for num in row]
    color_space = set(combined_list)

    output_puzzle = ARCPuzzle(output_grid)
    input_puzzle = ARCPuzzle(input_grid)

    Q = [(input_puzzle, 0)] # tuple of (ARCPuzzle, depth_int)
    parent_map = {input_puzzle: None} ## maps a ARCPuzzle to a tuple (ARCPuzzle, action)


    while len(Q) > 0:
        curr_puzzle, depth = Q.pop(0)
        if depth >= max_depth: break
        if curr_puzzle == output_puzzle: break ## sanity check. This line is only necessary if the input_grid == output_grid, which can't happen
        for action in curr_puzzle.legalActions(color_space):
            new_puzzle = curr_puzzle.copy()
            new_puzzle.executeAction(action)

            if new_puzzle in parent_map: continue

            parent_map[new_puzzle] = (curr_puzzle, action)
            Q.append((new_puzzle, depth + 1))
            if new_puzzle == output_puzzle:
                Q.clear() ## breaks outer loop
                break ## breaks inner loop

    ## work backwards to deduce the actions needed to transform input to output
    ## this will return an empty list if the BFS above did not find the output
    curr_puzzle = output_puzzle
    action_list = []
    while curr_puzzle in parent_map and parent_map[curr_puzzle] is not None:
        parent_puzzle, action = parent_map[curr_puzzle]
        action_list.insert(0, action)
        curr_puzzle = parent_puzzle

    return action_list


In [7]:
def execute_transformation(puzzle, trans):
    """
    Executes the given transformation on a grid and returns a 2D List
    """
    if not isinstance(puzzle, ARCPuzzle):
        puzzle = ARCPuzzle(puzzle)
    
    puzzle = puzzle.copy()
    
    for action in trans:
        puzzle.executeAction(action)
    
    return [list(row) for row in puzzle.state]

In [8]:
# Create an empty submission dict for output
submission = {}

# iterate over the test items and build up submission answers
count = 0
for key, task in tqdm(test_challenges.items()):

    # Here are the task's training inputs and outputs
    train_inputs = [item['input'] for item in task['train']]
    train_outputs = [item['output'] for item in task['train']]
    
    test_inputs = [item['input'] for item in task['test']]

    # Here we generate outputs for each test item.
    submission[key] = []

    transformations = set()
    for input_grid, output_grid in zip(train_inputs, train_outputs):
        transformations.add(tuple(arc_search(input_grid, output_grid, max_depth=2)))
    
    transformations = sorted(list(transformations), key=len, reverse=True) # longest solutions come first to avoid empty lists

    for input_grid in test_inputs:
        attempt_dict = dict()
        for i in range(2):
            trans = transformations[i % len(transformations)]
            soln = execute_transformation(input_grid, trans)
            attempt_dict[f'attempt_{i+1}'] = soln
            # if soln != input_grid:
            #     print("CHANGED SOMETHING!!")
        submission[key].append(attempt_dict)
            
# Here we write the submissions to file, so that they will get evaluated
with open('submission.json', 'w') as fp:
    json.dump(submission, fp)

100%|██████████| 400/400 [00:37<00:00, 10.73it/s]


Here is what our submission for the test task above looks like:

In [9]:
submission[sample_task]

[{'attempt_1': [[3, 2], [7, 8]], 'attempt_2': [[3, 2], [7, 8]]}]

# Scoring Your Submission

If you do not want to wait for gradescope to score your solution, we have provided the following code to score your submission. Note that the maximum possibe score is 400.

In [10]:
def score_submission():
    with open('../input/arc-prize-2024/arc-agi_evaluation_solutions.json', 'r') as sol_file:
        solutions = json.load(sol_file)
    
    with open('submission.json', 'r') as sub_file:
        submission = json.load(sub_file)
    
    overall_score = 0

    for task in solutions:
        score = 0
        for i, answer in enumerate(solutions[task]):
            attempt1_correct = submission[task][i]['attempt_1'] == answer
            attempt2_correct = submission[task][i]['attempt_2'] == answer
            score += int(attempt1_correct or attempt2_correct)

        if score > 0:
            print(json.dumps(test_challenges[task]))

        score /= len(solutions[task])

        overall_score += score
    

    print(overall_score)

You can run the above code by uncommenting the following code block. 

In [12]:
score_submission()

{"test": [{"input": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8], [0, 0, 0, 0, 0, 8, 0, 0, 0, 8, 0, 8, 0, 8], [0, 0, 0, 0, 0, 8, 0, 8, 0, 8, 0, 0, 0, 8], [0, 0, 0, 0, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}], "train": [{"input": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 8, 8, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 8, 0, 0, 0], [0, 0, 0, 0, 0, 8, 8, 8, 8, 0, 0, 0, 0, 0], [0, 0, 0, 0, 8, 8, 0, 8, 0, 0, 8, 8, 0, 0], [0, 0, 0, 0, 0, 0, 0, 8, 8, 8, 8, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 8, 0, 0, 0], [0, 0, 0, 0, 0, 0, 8, 8, 8, 0, 

# Confused about where to get started? 

If you're not sure what an initial solution might look like, then consider looking at public notebooks here: https://www.kaggle.com/competitions/arc-prize-2024/code or joining the public discussion here: https://www.kaggle.com/competitions/arc-prize-2024/discussion.

One example notebook that uses a very simple knowledge-based approach is this one: https://www.kaggle.com/code/michaelhodel/program-synthesis-starter-notebook/notebook, which conducts search over a space of domain specific block languages to form hypotheses and then applies these to test items.